# 1. import libraries

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as v_utils
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
import os

# 2. Data load

In [2]:
trans = transforms.Compose([transforms.Scale(size=100),
                      transforms.CenterCrop(64),
                       transforms.ToTensor()])

trainset = dset.ImageFolder(root='C:/Users/NHNEnt/Downloads/img_align_celeba',transform=trans)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,shuffle=True, num_workers=0,drop_last=True)

data_length = trainset.__len__()


# 3. Model

In [6]:
def deconv(ch_in,ch_out,k_size=4,stride=2,pad=1,bn=True):
    layers = []
    layers.append(nn.ConvTranspose2d(ch_in,ch_out,k_size,stride,padding=pad))
    if bn:
        layers.append(nn.BatchNorm2d(ch_out))
    return nn.Sequential(*layers)

def conv(ch_in, ch_out, k_size, stride, pad=1, bn=True):
    layers = []
    layers.append(nn.Conv2d(ch_in,ch_out,k_size,stride,padding=pad))
    if bn:
        layers.append(nn.BatchNorm2d(ch_out))
    return nn.Sequential(*layers)

In [8]:
class G(nn.Module):
    def __init__(self,dim_z=100,input_size=64,out_dim=128,colch=3,leaky=0.2):
        super(G,self).__init__()
        
        self.dim_z = dim_z
        self.input_size = input_size
        self.out_dim = out_dim
        self.colch = colch
        self.leaky = leaky
        
        self.fc = deconv(dim_z,out_dim*8,int(input_size/16),1,0,bn=False)
        self.deconv1 = deconv(out_dim*8,out_dim*4,4)
        self.deconv2 = deconv(out_dim*4,out_dim*2,4)
        self.deconv3 = deconv(out_dim*2,out_dim,4)
        self.deconv4 = deconv(out_dim,colch,4,bn=False)

        
    def forward(self,z):
        z = z.view(z.size(0),z.size(1),1,1)
        out = self.fc(z)
        out = F.leaky_relu(self.deconv1(out),self.leaky )
        out = F.leaky_relu(self.deconv2(out),self.leaky )
        out = F.leaky_relu(self.deconv3(out),self.leaky )
        out = F.tanh(self.deconv4(out))
        return out
        
class D(nn.Module):
    def __init__(self,input_size=64,conv_dim=128, colch=3):
        super(D,self).__init__()
        
        self.input_size = input_size
        self.colch = colch
        self.conv_dim = conv_dim
        
        self.conv1 = conv(self.colch,conv_dim,4,2,bn=False)
        self.conv2 = conv(conv_dim,conv_dim*2,4,2)
        self.conv3 = conv(conv_dim*2,conv_dim*4,4,2)
        self.conv4 = conv(conv_dim*4,conv_dim*8,4,2)
        self.fc = conv(conv_dim*8,1,4,1,0,bn=False)
        self.sigm = nn.Sigmoid()
                    
    def forward(self,x):
        out = F.relu(self.conv1(x))
        out = F.relu(self.conv2(out))
        out = F.relu(self.conv3(out))
        out = F.relu(self.conv4(out))
        out = self.fc(out)
        out = self.sigm(out).squeeze()
        return out

In [9]:
class DCGAN(object):
    def __init__(self,input_size=64,dim_z=100,colch=3,leaky=0.2,
                end_g_dim=64,start_d_dim=64):
        
        self.input_size= input_size
        self.dim_z = dim_z
        self.colch = colch
        self.leaky = leaky
        self.end_g_dim = end_g_dim
        self.start_d_dim = start_d_dim
        
        self.G = None
        self.D = None
        self.G_optim = None
        self.D_optim = None
        self.G_losses = []
        self.D_losses = []
        self.build_model()
        
        self.fixed_z = Variable(torch.rand(30,100)).cuda()
    def build_model(self):
        self.G = G(self.dim_z,self.input_size,self.end_g_dim,self.colch,self.leaky).cuda()
        self.D = D(self.input_size,self.start_d_dim, self.colch).cuda()
    
    def denorm(self, x):
        """Convert range (-1, 1) to (0, 1)"""
        out = (x + 1) / 2
        return out.clamp(0, 1)

    def train(self, data, batch_size=128,
              epoch=10,lr=0.0002,momentum = 0.5, check_step=100):
            
        batch_length = data.__len__()
        
        self.G_optim = torch.optim.SGD(self.G.parameters(),lr=lr,momentum = momentum)
        self.D_optim = torch.optim.SGD(self.D.parameters(),lr=lr,momentum = momentum)
        
        loss_func = nn.BCELoss()
        
        counter = 0
        
        if not os.path.isdir("./dcgan_result"):
            os.mkdir("./dcgan_result")

        for e in range(epoch):
            for i,[batch_x,_] in enumerate(data):

                batch_x = Variable(batch_x).cuda()
                # update D
                z = Variable(torch.rand(batch_size,self.dim_z)).cuda()
                
                self.D_optim.zero_grad()
                
                G_fake = self.denorm(self.G.forward(z))
                D_fake = self.D.forward(G_fake)
                D_real = self.D.forward(batch_x)
                
                D_loss = (torch.sum(loss_func(D_fake,Variable(torch.zeros(batch_size)).cuda()))+
                          torch.sum(loss_func(D_real,Variable(torch.ones(batch_size)).cuda())))

                D_loss.backward(retain_variables=False) # retain_variables=False : 다른 backward를 위해서              
                self.D_optim.step()
                
                # update G
                z = Variable(torch.rand(batch_size,self.dim_z)).cuda()
                
                self.G_optim.zero_grad()
                
                G_fake = self.denorm(self.G.forward(z))
                D_fake = self.D.forward(G_fake)
                
                G_loss = torch.sum(loss_func(D_fake,Variable(torch.ones(batch_size)).cuda()))
                
                G_loss.backward()              
                self.G_optim.step()
                
                counter+=1
                if counter % check_step == 0:
                    
                    self.D_losses.append(D_loss.cpu().data.numpy())
                    self.G_losses.append(G_loss.cpu().data.numpy())
                    
                    print("Epoch [%d/%d], Step [%d/%d], D_loss : %.4f, G_loss : %.4f"%(
                      e,epoch,i,batch_length,D_loss.cpu().data.numpy(),G_loss.cpu().data.numpy()))
                    
                    view = self.denorm(self.G.forward(self.fixed_z))
                    v_utils.save_image(view.data[0:25],"./dcgan_result/gen_{}_{}.png".format(e,i), nrow=5)


# 4.train

In [12]:
dcgan = DCGAN()

In [13]:
dcgan.train(trainloader,epoch=100,check_step=100)

C:\ProgramData\Anaconda3\lib\site-packages\torch\autograd\__init__.py:92: UserWarning: retain_variables option is deprecated and will be removed in 0.3. Use retain_graph instead.
  warnings.warn("retain_variables option is deprecated and will be removed in 0.3. "


Epoch [0/100], Step [99/1582], D_loss : 0.3473, G_loss : 2.0785
Epoch [0/100], Step [199/1582], D_loss : 0.3869, G_loss : 1.9724
Epoch [0/100], Step [299/1582], D_loss : 0.7200, G_loss : 1.5068
Epoch [0/100], Step [399/1582], D_loss : 0.8086, G_loss : 1.5355
Epoch [0/100], Step [499/1582], D_loss : 0.7496, G_loss : 1.5343
Epoch [0/100], Step [599/1582], D_loss : 0.4798, G_loss : 1.7561
Epoch [0/100], Step [699/1582], D_loss : 0.3919, G_loss : 1.8978
Epoch [0/100], Step [799/1582], D_loss : 0.5753, G_loss : 1.7096
Epoch [0/100], Step [899/1582], D_loss : 0.6907, G_loss : 1.6729
Epoch [0/100], Step [999/1582], D_loss : 0.4843, G_loss : 1.9273
Epoch [0/100], Step [1099/1582], D_loss : 0.5341, G_loss : 1.9187
Epoch [0/100], Step [1199/1582], D_loss : 0.5163, G_loss : 2.0403
Epoch [0/100], Step [1299/1582], D_loss : 0.4324, G_loss : 2.1354
Epoch [0/100], Step [1399/1582], D_loss : 0.2894, G_loss : 2.2495
Epoch [0/100], Step [1499/1582], D_loss : 0.4496, G_loss : 2.1117
Epoch [1/100], Step [

RuntimeError: Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\torch\utils\data\dataloader.py", line 40, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "C:\ProgramData\Anaconda3\lib\site-packages\torch\utils\data\dataloader.py", line 109, in default_collate
    return [default_collate(samples) for samples in transposed]
  File "C:\ProgramData\Anaconda3\lib\site-packages\torch\utils\data\dataloader.py", line 109, in <listcomp>
    return [default_collate(samples) for samples in transposed]
  File "C:\ProgramData\Anaconda3\lib\site-packages\torch\utils\data\dataloader.py", line 89, in default_collate
    storage = batch[0].storage()._new_shared(numel)
  File "C:\ProgramData\Anaconda3\lib\site-packages\torch\storage.py", line 111, in _new_shared
    return cls._new_using_filename(size)
RuntimeError: Couldn't open shared file mapping: <torch_5664_3245707374>, error code: <1455> at D:\Projects\pytorch\torch\lib\TH\TH